In [ ]:
from Bio import SeqIO
import pandas as pd

In [ ]:
annots = pd.read_csv('./sequence_db_africa_mbovis.csv')

In [ ]:
annots.head()

In [ ]:
with open ('./phylo_headers.txt', 'r') as file:
    headers = file.readlines()
    headers = [line.strip() for line in headers]

In [ ]:
# select sample in annots from headers

sample_annot = annots[annots['SRA ID'].isin(headers)]

In [ ]:
len(sample_annot)

In [ ]:
set_annots = sample_annot[['SRA ID', 'HOST', 'COUNTRY']]

In [ ]:
set_annots  # Display the first few rows of the selected annotations

In [ ]:
set_annots_nospace = set_annots.replace(' ', '_', regex=True)

In [ ]:
print(set_annots_nospace) # Display the first few rows of the selected annotations

In [ ]:
set_annots_nospace['HOST_COUNTRY'] = set_annots_nospace['HOST'] + '-' + set_annots_nospace['COUNTRY']

In [ ]:
print(set_annots_nospace)  # Display the updated DataFrame with HOST_COUNTRY

In [ ]:
mapping = dict(zip(set_annots_nospace['SRA ID'], set_annots_nospace['HOST_COUNTRY']))

In [ ]:
# Read, rename, and write sequences
sequences = []
name_counts = {}

for seq in SeqIO.parse("snps_filter3.aln", "fasta"):
    if seq.id != "Reference" and seq.id in mapping:
        new_name = mapping[seq.id]
        # Add counter for duplicates
        if new_name in name_counts:
            name_counts[new_name] += 1
            seq.id = f"{new_name}_{name_counts[new_name]}"
        else:
            name_counts[new_name] = 1
            seq.id = new_name
        seq.description = ""
    sequences.append(seq)

SeqIO.write(sequences, "snps_filter3_annot.aln", "fasta-2line")